Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline     
sns.set(color_codes=True)
# import requests
# import datetime

In [42]:
# Import data from file
df_ctybks = pd.read_csv('data/df_citybikes.csv')
df_fsq = pd.read_csv('data/df_fsq_all_loci102.csv')
# df_yelp = pd.read_csv('data/df_yelp_all_loci102.csv')

In [43]:
print(df_ctybks.shape)
print(df_fsq.shape)

(241, 10)
(2410, 15)


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [44]:
# Create a new column with the coordinate data we need, reformated to str dtype.

# Grab just the two columns we need lat and lon
df_ctybks.dropna(subset=['latitude', 'longitude'], inplace=True)

# Creation of a new variable with lon and lat together, label the column header 'lat-long'
df_ctybks['lat-long'] = df_ctybks['latitude'].astype(str) + ',' + df_ctybks['longitude'].astype(str)
df_ctybks = df_ctybks[df_ctybks['lat-long'] != '0.0,0.0']
df_ctybks.head()

,Station Name,slots,empty_slots,free_bikes,status,bank_card,latitude,longitude,timestamp,Unique ID,lat-long
0,0001 10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,1,"49.262487,-123.114397"
1,0004 Yaletown-Roundhouse Station,16,1,15,online,No,49.274566,-123.121817,2022-11-06T07:22:52.006000Z,4,"49.274566,-123.121817"
2,0005 Dunsmuir & Beatty,26,8,18,online,No,49.279764,-123.110154,2022-11-06T07:22:52.007000Z,5,"49.279764,-123.110154"
3,0007 12th & Yukon (City Hall),16,9,7,online,No,49.260599,-123.113504,2022-11-06T07:22:52.008000Z,7,"49.260599,-123.113504"
4,0008 8th & Ash,16,9,7,online,No,49.264215,-123.117772,2022-11-06T07:22:52.009000Z,8,"49.264215,-123.117772"


In [45]:
# Before merging, the lat & long columns can become confusing, so remove them first
df_ctybks = df_ctybks.drop(['latitude', 'longitude', 'Unique ID'], axis = 1)
df_ctybks.head(5)

,Station Name,slots,empty_slots,free_bikes,status,bank_card,timestamp,lat-long
0,0001 10th & Cambie,35,16,19,online,No,2022-11-06T07:22:52.004000Z,"49.262487,-123.114397"
1,0004 Yaletown-Roundhouse Station,16,1,15,online,No,2022-11-06T07:22:52.006000Z,"49.274566,-123.121817"
2,0005 Dunsmuir & Beatty,26,8,18,online,No,2022-11-06T07:22:52.007000Z,"49.279764,-123.110154"
3,0007 12th & Yukon (City Hall),16,9,7,online,No,2022-11-06T07:22:52.008000Z,"49.260599,-123.113504"
4,0008 8th & Ash,16,9,7,online,No,2022-11-06T07:22:52.009000Z,"49.264215,-123.117772"


In [47]:
# Does 'bank_card' or 'status' provide any meaningful data?
print(df_ctybks['bank_card'].unique())
print(df_ctybks['status'].unique())

['No']
['online' 'offline']


In [48]:
# Removing 'bank_card'
df_ctybks = df_ctybks.drop(['bank_card'], axis = 1)
df_ctybks.head(5)

,Station Name,slots,empty_slots,free_bikes,status,timestamp,lat-long
0,0001 10th & Cambie,35,16,19,online,2022-11-06T07:22:52.004000Z,"49.262487,-123.114397"
1,0004 Yaletown-Roundhouse Station,16,1,15,online,2022-11-06T07:22:52.006000Z,"49.274566,-123.121817"
2,0005 Dunsmuir & Beatty,26,8,18,online,2022-11-06T07:22:52.007000Z,"49.279764,-123.110154"
3,0007 12th & Yukon (City Hall),16,9,7,online,2022-11-06T07:22:52.008000Z,"49.260599,-123.113504"
4,0008 8th & Ash,16,9,7,online,2022-11-06T07:22:52.009000Z,"49.264215,-123.117772"


In [49]:
df_fsq.head(5)

,lat-long,fsq_id,cat_id,category_name,categories,name,distance,latitude,longitude,address,popularity,price,open_now,rating,rating_count
0,"49.262487,-123.114397",4fd3a01be4b0540d9cb027ea,13034.0,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Moii Cafe,57,49.262562,-123.115171,"2559 Cambie St (btwn W Broadway & W 10th Ave),...",0.221702,1.0,False,8.0,55.0
1,"49.262487,-123.114397",4b1db335f964a520211424e3,13034.0,Café,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",Marulilu Cafe,96,49.263364,-123.113942,"451W Broadway W (at Cambie St), Vancouver BC V...",0.953849,2.0,False,8.2,111.0
2,"49.262487,-123.114397",4db0b65d4df03036e8c0960c,11064.0,Hair Salon,"[{'id': 11064, 'name': 'Hair Salon', 'icon': {...",Space Salon,128,49.263102,-123.115785,"530W Broadway W (btw Ash & Cambie), Vancouver ...",0.785993,NaN,False,8.3,12.0
3,"49.262487,-123.114397",4bc290d674a9a593788ed3f6,11046.0,Banking and Finance,"[{'id': 11046, 'name': 'Banking and Finance', ...",Vancity Savings Credit Union,48,49.262392,-123.115157,"501 10th Ave W, Vancouver BC V5Z 1K9",0.944987,NaN,False,7.2,8.0
4,"49.262487,-123.114397",4e223f06d4c0d32590f80ff4,13306.0,Taco Restaurant,"[{'id': 13306, 'name': 'Taco Restaurant', 'ico...",La Taqueria Pinche Taco Shop,164,49.263601,-123.112665,"2450 Yukon St (at W Broadway), Vancouver BC V5...",0.954043,2.0,True,8.7,367.0


In [25]:
df_merged = pd.merge(df_ctybks, df_fsq, how='inner', on='lat-long')
print(df_merged.shape)
df_merged.head(5)

(2410, 26)


,Unnamed: 0,Station Name,slots,empty_slots,free_bikes,status,bank_card,latitude_x,longitude_x,timestamp,...,name,distance,latitude_y,longitude_y,address,popularity,price,open_now,rating,rating_count
0,0,0001 10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,...,Moii Cafe,57,49.262562,-123.115171,"2559 Cambie St (btwn W Broadway & W 10th Ave),...",0.221702,1.0,False,8.0,55.0
1,0,0001 10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,...,Marulilu Cafe,96,49.263364,-123.113942,"451W Broadway W (at Cambie St), Vancouver BC V...",0.953849,2.0,False,8.2,111.0
2,0,0001 10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,...,Space Salon,128,49.263102,-123.115785,"530W Broadway W (btw Ash & Cambie), Vancouver ...",0.785993,NaN,False,8.3,12.0
3,0,0001 10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,...,Vancity Savings Credit Union,48,49.262392,-123.115157,"501 10th Ave W, Vancouver BC V5Z 1K9",0.944987,NaN,False,7.2,8.0
4,0,0001 10th & Cambie,35,16,19,online,No,49.262487,-123.114397,2022-11-06T07:22:52.004000Z,...,La Taqueria Pinche Taco Shop,164,49.263601,-123.112665,"2450 Yukon St (at W Broadway), Vancouver BC V5...",0.954043,2.0,True,8.7,367.0


In [27]:
# Checking the types of data
df = df_merged.copy()
df.dtypes

Unnamed: 0         int64
Station Name      object
slots              int64
empty_slots        int64
free_bikes         int64
status            object
bank_card         object
latitude_x       float64
longitude_x      float64
timestamp         object
Unique ID          int64
lat-long          object
fsq_id            object
cat_id           float64
category_name     object
categories        object
name              object
distance           int64
latitude_y       float64
longitude_y      float64
address           object
popularity       float64
price            float64
open_now            bool
rating           float64
rating_count     float64
dtype: object

In [32]:
# Dropping irrelevant columns
df = df.drop(['categories', 'Unique ID'], axis=1)

In [31]:
# Recheck headings:
df.dtypes

Unnamed: 0         int64
Station Name      object
slots              int64
empty_slots        int64
free_bikes         int64
status            object
bank_card         object
latitude_x       float64
longitude_x      float64
timestamp         object
lat-long          object
fsq_id            object
cat_id           float64
category_name     object
name              object
distance           int64
latitude_y       float64
longitude_y      float64
address           object
popularity       float64
price            float64
open_now            bool
rating           float64
rating_count     float64
dtype: object

In [ ]:
# Dropping irrelevant columns
# df = df.drop(['col_a', 'col_b', 'col_c_etc.'], axis=1)

# Recheck headings:
# df.head(5)

In [ ]:
# Renaming the columns if needed - say if you wanted them "short and pithy Dr B!"
# df = df.rename(columns = { 'Lengthy_current':'LP', 'doggy_booty_toot':'Lucy_etc.' })
# df.head(5)

In [ ]:
# Checking for the presence and location of duplicate rows
# duplicate_rows_df = df[df.duplicated()]

# And compare the difference in the DF shape:
# print(df.shape)
# print("number of duplicate rows: ", duplicate_rows_df.shape)

In [ ]:
# Assess the location and content of the duplications, are they appropriate?
# If not, we can drop them:

# First a count of the total rows to compare to #of duplicate rows
# df.count()

# Then we drop, and recheck:
# df = df.drop_duplicates()
# df.head(5)

In [ ]:
# Drop any missing or null values:
# print(df.isnull().sum())

In [ ]:
# We'll drop the full rows that contain null values:
# df = dfdropna()
# df.count()

In [ ]:
# Validating the dropped missing values
# print(df.isnull().sum())

In [ ]:
# Detecting outliers on numerical values
# sns.boxplot(x=df['col_with_numerical_value'])

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.